In [14]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

MAXG=5000
PSIZE=50
D = 10

PM = 0.01
PC = 0.65

ACKLEY1 = 1
ACKLEY2 = 2
ACKLEY3 = 3
ACKLEY4 = 4

'''subject to −35 ≤ xi ≤ 35. The global minima is located at origin x∗ = (0, · · · , 0),
f(x∗) = 0.'''
def ackley_1(individuo):
    global D
    sum1 = 0.0
    sum2 = 0.0
    for g in individuo:
        sum1 = sum1 + g**2.0
        sum2 = sum2 + np.cos(2.0*np.pi*g)
    return -20.0*np.exp(-0.02*np.sqrt(sum1/D)) - np.exp(sum2/D) + 20 + np.e


'''subject to −32 ≤ xi ≤ 32. The global minimum is located at origin x∗ = (0, 0),
f(x∗) = −200. '''  
def ackley_2(individuo):
    return -200*np.exp(-0.02*np.sqrt(individuo[0]**2+individuo[1]**2))


'''subject to −32 ≤ xi ≤ 32. The global minimum is located at x∗ = (0, ≈ −0.4),
f(x∗) ≈ −219.1418.'''
def ackley_3(individuo):
    return ackley_2(individuo)+5*np.exp(np.cos(3*individuo[0])+np.sin(3*individuo[1]))


'''subject to −35 ≤ xi ≤ 35. It is highly multimodal function with two global minimum
close to origin
x = f({−1.479252, −0.739807}, {1.479252, −0.739807}), f(x∗) = −3.917275'''
def ackley_4(individuo):
    global D
    suma = 0.0
    i=0
    while (i+1 < D):
        suma = suma + np.exp(-0.2)*np.sqrt(individuo[i]**2+individuo[i+1]**2)
        +3*(np.cos(2*individuo[i])+np.sin(2*individuo[i+1]))
        i = i + 1
    return suma

def init_populations(pop_size = PSIZE):
    global D
    pop_1 = []
    pop_2 = []
    pop_3 = []
    pop_4 = []
    for i in range(pop_size):
        pop_1.append([random.uniform(-35, 35) for i in range(D)])
        pop_2.append([random.uniform(-32, 32), random.uniform(-32, 32)])
        pop_3.append([random.uniform(-32, 32), random.uniform(-32, 32)])
        pop_4.append([random.uniform(-35, 35) for i in range(D)])
    return [np.array(pop_1),np.array(pop_2),np.array(pop_3),np.array(pop_4)]

def calculate_fitness(pop,ackley):
    switch = {
        1: ackley_1,
        2: ackley_2,
        3: ackley_3,
        4: ackley_4
    }
    scores = []
    for ind in pop:
        scores.append(switch.get(ackley)(ind))
    return scores

def select_individual_by_tournament(population, scores):
    population_size = len(scores)
    # Pick individuals for tournament
    fighter_1 = random.randint(0, population_size-1)
    fighter_2 = random.randint(0, population_size-1)
    while(fighter_1==fighter_2):
        fighter_2=random.randint(0,population_size-1)
    # Get fitness score for each
    fighter_1_fitness = scores[fighter_1]
    fighter_2_fitness = scores[fighter_2]
    # Identify undividual with highest fitness
    # Fighter 1 will win if score are equal
    if fighter_1_fitness <= fighter_2_fitness:
        winner = fighter_1
    else:
        winner = fighter_2
    # Return the chromsome of the winner
    return population[winner, :]

def crossover(parent_1,parent_2):
    chromosome_length = len(parent_1)
    child1 = parent_1[:]
    child2 = parent_2[:]
    if chromosome_length == 2:
        child1[1] = parent_2[1]
        child2[1] = parent_1[1]
    else:
        alfa = random.random()
        pivote = math.floor((chromosome_length-1)*alfa)
        while pivote==0 or pivote == chromosome_length-1: #Esto en teoria nunca pasa
            alfa = random.random()
            pivote = math.floor((chromosome_length-1)*alfa)
        child1[pivote:] = parent_2[pivote:]
        child2[pivote:] = parent_1[pivote:]
    return child1, child2

def mutation(ind):
    chromosome_length = len(ind)
    i=random.randint(0,chromosome_length-1)
    if chromosome_length == 2:
        b = [-32,32]
    else:
        b = [-35,35]
    if random.random() < 0.5:
        ind[i] = ind[i] + random.random()*(b[1]-ind[i])
    else:
        ind[i] = ind[i] - random.random()*(ind[i]-b[0])
    return ind

def ejecucion_ga(pop,ackley,scores,best_score,best_scores_progress,seed):
    random.seed(seed)
    np.random.seed(seed)
    mejor_puntaje = float('inf')
    mejor_solucion = []
    for generation in range(MAXG):
        new_population=[]
        if best_score < mejor_puntaje:
            mejor_puntaje = best_score
            mejor_solucion = pop[np.argmin(scores)][:]
        for i in range(int(PSIZE/2)):
            parent_1 = select_individual_by_tournament(pop, scores)
            parent_2 = select_individual_by_tournament(pop, scores)
            #Evolucion
            if np.random.random()<PC:
                child_1, child_2 = crossover(parent_1, parent_2)
            else:
                child_1=parent_1
                child_2=parent_2
            #Mutacion
            if np.random.random()<PM:
                mutation(child_1)
            if np.random.random()<PM:
                mutation(child_2)
            new_population.append(child_1)
            new_population.append(child_2)
        pop=np.array(new_population)
        scores=calculate_fitness(pop,ackley)
        best_score=np.min(scores)
        best_scores_progress.append(best_score)
    return [mejor_puntaje, mejor_solucion]
 
pop = init_populations()
best_scores_progress = []
scores = []
repeticiones = 25
estadisticas = []

for i in range(4):
    progreso_mejores = []
    mejor_actual = float('inf')
    mejor_sol = []
    scores = calculate_fitness(pop[i],i+1) 
    for j in range(repeticiones):
        resultados = ejecucion_ga(i+1,pop[i],scores,best_score,best_scores_progress,j)
        if(float(resultados[0]) < mejor_actual):
            mejor_actual = resultados[0]
            mejor_sol = resultados[1]
        #print(resultados[1])#ACAME TRAE LA MEJOR SOLUCION QUE QUIERO 
        progreso_mejores.append(resultados[0])
    #Sin Rotacion
    media = np.average(progreso_mejores)
    mediana = np.median(progreso_mejores)
    desviacion = np.std(progreso_mejores)    
    
    
'''    
pop1, pop2, pop3, pop4 = init_populations()
scores = []
best_score = 0
best_scores_progress = []
scores = calculate_fitness(pop1,ACKLEY1)
solution = ejecucion_ga(pop1,ACKLEY1,scores,best_score,best_scores_progress)
print('\nSolution: f(%s) = %.5f' % (np.around(solution[1], decimals=2), solution[0]))
plt.plot(best_scores_progress, '.-')
plt.xlabel('Generacion Numero')
plt.ylabel('f(x)')
plt.show()
print('---')

best_scores_progress = []
scores = calculate_fitness(pop2,ACKLEY2)
solution = ejecucion_ga(pop2,ACKLEY2,scores,best_score,best_scores_progress)
print('\nSolution: f(%s) = %.5f' % (np.around(solution[1], decimals=5), solution[0]))
plt.plot(best_scores_progress, '.-')
plt.xlabel('Generacion Numero')
plt.ylabel('f(x)')
plt.show()
print('---')

best_scores_progress = []
scores = calculate_fitness(pop3,ACKLEY3)
solution = ejecucion_ga(pop3,ACKLEY3,scores,best_score,best_scores_progress)
print('\nSolution: f(%s) = %.5f' % (np.around(solution[1], decimals=5), solution[0]))
plt.plot(best_scores_progress, '.-')
plt.xlabel('Generacion Numero')
plt.ylabel('f(x)')
plt.show()
print('---')

best_scores_progress = []
scores = calculate_fitness(pop4,ACKLEY4)
solution = ejecucion_ga(pop4,ACKLEY4,scores,best_score,best_scores_progress)
print('\nSolution: f(%s) = %.3f' % (np.around(solution[1], decimals=2), solution[0]))
plt.plot(best_scores_progress, '.-')
plt.xlabel('Generacion Numero')
plt.ylabel('f(x)')
plt.show()'''

TypeError: 'int' object is not subscriptable